In [1]:
######importing libraries for data manipulation#######
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gzip
import math
from PIL import Image
import requests
from io import BytesIO
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import fbeta_score,precision_score,f1_score,recall_score,accuracy_score
from sklearn.linear_model import Perceptron
from spark_sklearn import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
import mord
from time import time
import matplotlib.patches as mpatches
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [3]:
###############Function definitions###########

#########Function definitions for separating related feature########
def change_vals_new_col(s,value,new_cols):
    
    if(value.get(s) != None):
        if((type(value[s]) == float) and np.isnan(value[s])):
            new_cols[s] = np.nan
        else:
            new_cols.get(s).append(value[s])
    else:
        new_cols.get(s).append(np.nan)
    

def generate_new_cols(related):
    
    new_cols = {'also_bought':[], 'also_viewed':[],'bought_together':[],'buy_after_viewing':[]}
    
    for key,value in related.items():
        if((type(value) == float) and np.isnan(value)):
            
            new_cols['also_bought'].append(np.nan)
            new_cols['also_viewed'].append(np.nan)
            new_cols['bought_together'].append(np.nan)
            new_cols['buy_after_viewing'].append(np.nan)
        else:
            change_vals_new_col('also_bought',value,new_cols)
            change_vals_new_col('also_viewed',value,new_cols)
            change_vals_new_col('bought_together',value,new_cols)
            change_vals_new_col('buy_after_viewing',value,new_cols)
            
        
    return new_cols
#####Function definitions for separating related feature ends####

def plot_related_prods(related,which,final_metadata):
    
    if(related == None):
        print('People who'+str(related)+'this product did not buy any other product:')
        return
    else:
        #print(np.array(related) in final_metadata.index)
        tot = 0
        for idx in related:
            if(idx in final_metadata.index):
                tot += 1
        print(tot)
        tot = round(tot/2)
        print('final',tot)
        f, axes = plt.subplots(tot,tot,figsize=(4,4),dpi=300)
        f.suptitle('People also '+str(which))
        for i in range(0,tot):
            for j in range(0,tot):
                curr_asin = related[i+j]
                if((curr_asin in final_metadata.index) == True):
                    curr_url = final_metadata.loc[curr_asin]['imUrl']
                    curr_title = final_metadata.loc[curr_asin]['title']
                    curr_title = curr_title[0:30]
                    response = requests.get(curr_url)
                    img = Image.open(BytesIO(response.content))
                    axes[i,j].imshow(img)
                    axes[i,j].get_xaxis().set_ticks([])
                    axes[i,j].get_yaxis().set_ticks([])
                    plt.axis('off')
                    axes[i,j].set_title(curr_title,size=3)
        plt.show()            

def Show_related_products(meta_data_row,final_metadata):
    
    #print(meta_data_row)
    curr_url = meta_data_row['imUrl']
    #curr_prod_id = meta_data_row['asin']
    title = meta_data_row['title']
    
    print('The current product is:',title)
    response = requests.get(curr_url)
    img = Image.open(BytesIO(response.content))
    plt.imshow(img)
    plt.show()
    
    ####People who bought this product also bought####
    also_bought = meta_data_row['also_bought']
    if((type(also_bought) == float) and np.isnan(also_bought)):
        also_bought = None
    else:
        if(len(also_bought) > 9):
            also_bought = also_bought[0:9]
    plot_related_prods(also_bought,'bought',final_metadata)
    
    ####People who bought this product also viewed####
    also_viewed = meta_data_row['also_viewed']
    if((type(also_viewed) == float) and np.isnan(also_viewed)):
        also_viewed = None
    else:
        if(len(also_viewed) > 9):
            also_viewed = also_viewed[0:9]
    plot_related_prods(also_viewed,'viewed',final_metadata)


    

# Read Data

In [4]:
#####Reading Data##ata#####
#####This is a smaller data for initial data exploration and model testing#####
######The data is about Heal and Personal care Products on Amazon##########

def parse(path):
    g = open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

data_path = '/Users/ruchinpatel/Desktop/USC_EVERYTHING/SPRINGBOARD/CAPSTONE/Health_and_Personal_Care_5.json'
metadata_path = '/Users/ruchinpatel/Desktop/USC_EVERYTHING/SPRINGBOARD/CAPSTONE/meta_Health_and_Personal_Care.json'

data = getDF(data_path)
metadata = getDF(metadata_path)

In [5]:
########## Generating seperate columns for related feature######
related = metadata['related'].to_dict()
newly_created_columns = pd.DataFrame(generate_new_cols(related))

In [6]:
#########Final Metadata dataframe###########
final_metadata = pd.concat([metadata,newly_created_columns],axis = 1)
final_metadata = final_metadata.drop('related',axis=1)
final_metadata = final_metadata.set_index('asin')

######converting the dates to date time format#####
data['unixReviewTime'] = pd.to_datetime(data['unixReviewTime'],unit='s')
data['reviewTime'] = pd.to_datetime(data['reviewTime'])

In [7]:
data.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ALC5GH8CAMAI7,159985130X,AnnN,"[1, 1]",This is a great little gadget to have around. ...,5.0,Handy little gadget,2011-01-05,2011-01-05
1,AHKSURW85PJUE,159985130X,"AZ buyer ""AZ buyer""","[1, 1]",I would recommend this for a travel magnifier ...,4.0,Small & may need to encourage battery,2012-02-18,2012-02-18
2,A38RMU1Y5TDP9,159985130X,"Bob Tobias ""Robert Tobias""","[75, 77]",What I liked was the quality of the lens and t...,4.0,Very good but not great,2010-06-08,2010-06-08
3,A1XZUG7DFXXOS4,159985130X,Cat lover,"[56, 60]",Love the Great point light pocket magnifier! ...,4.0,great addition to your purse,2008-02-08,2008-02-08
4,A1MS3M7M7AM13X,159985130X,Cricketoes,"[1, 1]",This is very nice. You pull out on the magnifi...,5.0,Very nice and convenient.,2011-08-16,2011-08-16
5,AXO4PQU0XG3TG,159985130X,Dwight,"[2, 3]",The light comes on when the item is pulled. T...,5.0,"$9.99, pretty and cute",2007-02-24,2007-02-24
6,A28X0LT2100RL1,159985130X,Eusebius,"[0, 0]",These are lightweight and efficient and have s...,4.0,Lightweight and efficient,2014-07-06,2014-07-06
7,A1VUSWRVN8SJA8,159985130X,"Gary ""Gary""","[2, 2]",We bought one for road trips and trying to int...,5.0,Excellent Design and Functionaality,2011-02-24,2011-02-24
8,A1JQDCX4LDKBZ3,159985130X,guru,"[1, 1]",The screen of the magnifier is small. If you'r...,3.0,Okay,2013-01-24,2013-01-24
9,A3RNRXOM5J2C93,159985130X,Jeffrey E. Cornett,"[1, 1]",This pocket magnifier is nice and compact. Th...,4.0,Pocket magnifier,2012-06-02,2012-06-02


In [8]:
metadata.head(10)

,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN
1,0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,NaN,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN
2,0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,NaN
3,0615315860,NaN,101 BlenderBottle Recipes Quick and Easy,http://ecx.images-amazon.com/images/I/21zOQu2Q...,"{'also_bought': ['B006VT9RBM', 'B0010JLMO8', '...",{'Health & Personal Care': 254068},[[Health & Personal Care]],NaN,NaN
4,0615406394,This is an example product description.,"Aphrodite Reborn - Women's Stories of Hope, Co...",http://ecx.images-amazon.com/images/I/51rJLgsi...,"{'also_bought': ['0966035232', '1421407205']}",{'Health & Personal Care': 377936},[[Health & Personal Care]],NaN,NaN
5,0615836828,NaN,Breast Cancer: My Spiritual Journey,http://ecx.images-amazon.com/images/I/41UrGEwA...,NaN,{'Health & Personal Care': 603899},[[Health & Personal Care]],NaN,NaN
6,0641710577,Foster Grant introduces a whole new dimension ...,Foster Grant Women's Reading Glasses Romance W...,http://ecx.images-amazon.com/images/I/414Fx6sG...,NaN,{'Health & Personal Care': 1039561},"[[Health & Personal Care, Personal Care, Eye C...",NaN,NaN
7,0641864507,* Super bright LED light\n * 2 CR2016 repla...,LED Travel Magnifier,http://ecx.images-amazon.com/images/I/41ZlfiUd...,"{'also_viewed': ['B002E3KJBG', '1935009656', '...",NaN,"[[Health & Personal Care, Medical Supplies & E...",NaN,UltraOptix
8,0681504498,"Affordably priced. Hardcover or paper back, th...",Itty Bitty Paperback Booklight,http://ecx.images-amazon.com/images/I/31pQu4fW...,"{'also_viewed': ['1933622717', 'B00005R8TK'], ...",NaN,"[[Health & Personal Care, Medical Supplies & E...",21.97,Maxi-Aids
9,0705394638,Beautiful memory book for you new baby boy to ...,C.r. Gibson Baby Boy Zoom-Zoom Loose-leaf Memo...,http://ecx.images-amazon.com/images/I/51hf80nu...,"{'also_viewed': ['B00AYZFKME', 'B001AL7GOQ', '...",{'Health & Personal Care': 72478},"[[Health & Personal Care, Stationery & Party S...",NaN,NaN


In [9]:
users = data.reviewerID.unique().shape[0]
items = metadata.asin.unique().shape[0]

In [10]:
print(users)
print(items)

38609
263032


In [11]:
for line in data.head(10).itertuples():
    print(line[1])

ALC5GH8CAMAI7
AHKSURW85PJUE
A38RMU1Y5TDP9
A1XZUG7DFXXOS4
A1MS3M7M7AM13X
AXO4PQU0XG3TG
A28X0LT2100RL1
A1VUSWRVN8SJA8
A1JQDCX4LDKBZ3
A3RNRXOM5J2C93


In [12]:
r_index = data.reviewerID.unique()
c_index = data.asin.unique()

In [13]:
d = pd.DataFrame(0, index=r_index, columns=c_index)

In [14]:
for line in data.itertuples():
    d.ix[line[1]].loc[line[2]] = int(line[6])

In [32]:
np.unique(d)

array([0, 1, 2, 3, 4, 5])

In [15]:
d.head(10)

,159985130X,1933622865,1935009656,3812028492,7884890364,B00000J47L,B00000J9DU,B00000JHQ2,B00000JHQ6,B00000JHQC,...,B00L5L7ZPQ,B00L5LSO7E,B00L6FWTJ8,B00L9BRHGY,B00LBE8SMG,B00LDUS8H8,B00LFB2ENY,B00LIASY0Y,B00LJBMCFA,B00LJBMCKK
ALC5GH8CAMAI7,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AHKSURW85PJUE,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A38RMU1Y5TDP9,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1XZUG7DFXXOS4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1MS3M7M7AM13X,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AXO4PQU0XG3TG,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A28X0LT2100RL1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1VUSWRVN8SJA8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1JQDCX4LDKBZ3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3RNRXOM5J2C93,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(d, metric='cosine')
item_similarity = pairwise_distances(d.T, metric='cosine')